In [1]:
# Read in Subway data from API 
import pandas as pd
import urllib.parse
from io import StringIO
from data_utils import establish_nys_session

In [2]:
def get_data_using_query(soql_query):
    # The base part of the URL
    base_url = "https://data.ny.gov/resource/wujg-7c2s.csv?$query="
    
    # Encode the SQL query to be URL-safe
    encoded_query = urllib.parse.quote(soql_query)
    
    # Combine the base URL with the encoded query
    full_url = base_url + encoded_query
    
    session = establish_nys_session()
    response = session.get(full_url)
    
    if response.status_code == 200:
        print("Request was successful!")
    else:
        print("Request failed with status code:", response.status_code)
        
    # Convert the CSV response to a DataFrame
    data = StringIO(response.text)
    df = pd.read_csv(data, header=0)
    return df

# Ridership data

In [3]:
# Your SOQL query
start_date = '2022-12-31' #year, month, date
end_date = '2023-01-05' #year, month, date

ridership_data_query = f"""
SELECT 
    date_trunc_ymd(transit_timestamp) AS date, 
    station_complex, 
    borough,
    payment_method,
    round(SUM(ridership),0) AS total_ridership,
    MAX(latitude) AS lat, 
    MAX(longitude) AS lng
WHERE (transit_timestamp >= '{start_date}T00:00:00') AND (transit_timestamp < '{end_date}T00:00:00')
GROUP BY 
    date, 
    station_complex, 
    borough,
    payment_method
ORDER BY 
    date ASC
LIMIT 1000000
""".replace("\n", " ")  # Remove newlines for a single line query

ridership_data = get_data_using_query(ridership_data_query)

Request was successful!


In [4]:
ridership_data['date'] = pd.to_datetime(ridership_data['date'])

In [5]:
ridership_data['date'].unique()

<DatetimeArray>
['2022-12-31 00:00:00', '2023-01-01 00:00:00', '2023-01-02 00:00:00',
 '2023-01-03 00:00:00', '2023-01-04 00:00:00']
Length: 5, dtype: datetime64[ns]

In [6]:
ridership_data

,date,station_complex,borough,payment_method,total_ridership,lat,lng
0,2022-12-31,103 St (1),Manhattan,metrocard,37,40.799446,-73.968376
1,2022-12-31,103 St (1),Manhattan,omny,34,40.799446,-73.968376
2,2022-12-31,103 St (6),Manhattan,metrocard,23,40.790600,-73.947479
3,2022-12-31,103 St (6),Manhattan,omny,9,40.790600,-73.947479
4,2022-12-31,"103 St (C,B)",Manhattan,metrocard,7,40.796093,-73.961456
...,...,...,...,...,...,...,...
4226,2023-01-04,WTC Cortlandt (1),Manhattan,omny,4002,40.711834,-74.012192
4227,2023-01-04,York St (F),Brooklyn,metrocard,3843,40.701397,-73.986748
4228,2023-01-04,York St (F),Brooklyn,omny,4168,40.701397,-73.986748
4229,2023-01-04,Zerega Av (6),Bronx,metrocard,1145,40.836487,-73.847038
